In [89]:
import glob
import os
import re

txt_file_path = 'data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:30])

데이터 크기: 187088
Examples:
 ['', '', 'All of this and more is for you', 'With love, sincerity and deepest care', 'My life with you I share', '', '', 'Ever since I met you, baby', "I've been wantin' to lay you down", "But it's so hard to get you", 'Baby, when you never come around', 'Every day that you keep it away', 'It only makes me want it more', 'Ooh baby, just say the word', "And I'll be at your door", '', 'What more do I have to say?', 'I really wanna play in your river', '', "Fallin', fallin', fallin' in love", "I'm fallin', baby, deeper every day", '(In love)', "You're breakin' my heart and takin' me away", '(In love)', "I'm fallin' baby, girl, what can I do?", "I just can't be without you", '', 'Ever since I met you, baby', "There's been somethin' inside of me", "That keeps me wantin' you"]


In [90]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [91]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    size = len(sentence)
    # 우리가 원하지 않는 문장은 건너뜁니다
    if size == 0: continue
    if len(sentence.split()) > 15: continue

    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> all of this and more is for you <end>',
 '<start> with love , sincerity and deepest care <end>',
 '<start> my life with you i share <end>',
 '<start> ever since i met you , baby <end>',
 '<start> i ve been wantin to lay you down <end>',
 '<start> but it s so hard to get you <end>',
 '<start> baby , when you never come around <end>',
 '<start> every day that you keep it away <end>',
 '<start> it only makes me want it more <end>',
 '<start> ooh baby , just say the word <end>']

In [94]:
import tensorflow as tf
import numpy as np

# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')


    # print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

In [95]:
print(tensor[:3, :10])

[[    2    25    19    42     8    99    26    28     7     3]
 [    2    31    37     4 10907     8  4580   283     3     0]
 [    2    13   103    31     7     5  1011     3     0     0]]


In [96]:
from sklearn.model_selection import train_test_split
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]

# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, random_state=42, test_size=0.2)

print(enc_train.shape, enc_val.shape, dec_train.shape, dec_val.shape)

(134872, 32) (33718, 32) (134872, 32) (33718, 32)


In [97]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
# dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [98]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out

embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [99]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[ 2.0339833e-04, -1.3113962e-04,  1.8605235e-04, ...,
          2.0805551e-04, -1.2453410e-04,  1.7848083e-04],
        [ 3.1596981e-04, -1.8448906e-04,  4.2919844e-04, ...,
          3.5026073e-04, -8.1933598e-05,  1.2390864e-04],
        [ 4.3627911e-04, -3.5693723e-04,  6.4603641e-04, ...,
          2.7645507e-04, -1.6237222e-04, -7.7348450e-05],
        ...,
        [ 8.0732588e-04, -1.7483747e-03, -6.0210860e-04, ...,
          5.5146147e-03,  9.0605096e-04, -2.0381466e-03],
        [ 8.4172311e-04, -1.7660458e-03, -5.9560913e-04, ...,
          5.5709886e-03,  9.4084523e-04, -2.0655540e-03],
        [ 8.7288272e-04, -1.7842633e-03, -5.8699428e-04, ...,
          5.6134318e-03,  9.6913829e-04, -2.0857281e-03]],

       [[ 2.0339833e-04, -1.3113962e-04,  1.8605235e-04, ...,
          2.0805551e-04, -1.2453410e-04,  1.7848083e-04],
        [ 1.5798901e-04,  5.3477863e-05,  4.8102523e-04, ...,
          2.6325547e-04, 

In [100]:
model.summary()

Model: "text_generator_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_12 (LSTM)               multiple                  5246976   
_________________________________________________________________
lstm_13 (LSTM)               multiple                  8392704   
_________________________________________________________________
dense_6 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [101]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30


NotImplementedError: in user code:

    /Users/me/miniforge3/envs/hongong/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    <ipython-input-66-8cc0fe026c72>:12 call  *
        out = self.rnn_1(out)
    /Users/me/miniforge3/envs/hongong/lib/python3.9/site-packages/tensorflow/python/keras/layers/recurrent.py:660 __call__  **
        
    /Users/me/miniforge3/envs/hongong/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py:1012 __call__
        
    /Users/me/miniforge3/envs/hongong/lib/python3.9/site-packages/tensorflow/python/keras/layers/recurrent_v2.py:1157 call
        
    /Users/me/miniforge3/envs/hongong/lib/python3.9/site-packages/tensorflow/python/keras/layers/recurrent.py:859 _process_inputs
        
    /Users/me/miniforge3/envs/hongong/lib/python3.9/site-packages/tensorflow/python/keras/layers/recurrent.py:642 get_initial_state
        
    /Users/me/miniforge3/envs